# RailTrack Cloud | Stage 1: Dataset Simulation

This notebook generates **synthetic railcar operation data** for use in later Azure + Power BI pipeline stages.  
It represents the first step in the **RailTrack Cloud Data Engineering Project**, which demonstrates how data moves from local simulation → Azure Blob → Azure SQL → Power BI dashboard.

---

**Objectives**
- Simulate realistic daily railcar data  
- Prepare dataset for Azure ingestion  
- Validate and export as CSV for pipeline use

**Generated Features**
| Column | Description |
|:--------|:-------------|
| `date` | Calendar date of record |
| `railcar_id` | Unique railcar identifier |
| `distance_km` | Daily distance traveled (km) |
| `fuel_used_l` | Fuel used (liters) |
| `temperature_c` | Average operating temperature (°C) |
| `maintenance_flag` | 1 = maintenance performed, 0 = normal |


In [11]:
# ===============================================================
# RAILTRACK CLOUD - DATASET SIMULATION
# ---------------------------------------------------------------
# Purpose: Generate synthetic railcar operation data for use in
#          later Azure + Power BI pipeline stages.
# ===============================================================

# --- 1. Import Libraries ---
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os

## Define Simulation Parameters

We’ll specify the number of railcars, total simulation days, and the starting date for data generation.

In [12]:
# --- 2. Simulation Parameters ---
num_railcars = 15       # total number of railcars
num_days = 60           # number of days to simulate
start_date = datetime(2025, 1, 1)   # start date for simulation

## Create Railcar ID List

Each railcar receives a unique ID in the format `RC001`, `RC002`, etc.

In [13]:
# --- 3. Create Railcar ID List ---
railcar_ids = [f"RC{str(i).zfill(3)}" for i in range(1, num_railcars + 1)]

## Generate Daily Data Records

For each railcar and each simulated day:
- Distance is randomized between 50–200 km  
- Fuel consumption ≈ 0.6–0.8 L/km  
- Temperature fluctuates between 15–40°C  
- Maintenance occurs with ~3% probability

In [14]:
# --- 4. Generate Data ---
data = []

for day in range(num_days):
    date = start_date + timedelta(days=day)
    for rc in railcar_ids:
        distance = round(random.uniform(50, 200), 2)  # km traveled
        fuel_used = round(distance * random.uniform(0.6, 0.8), 2)  # fuel ≈ 0.6–0.8 L/km
        temp = round(random.uniform(15, 40), 1)       # avg temperature
        maintenance_flag = 1 if random.random() < 0.03 else 0  # ~3 % chance of maintenance

        data.append([date.date(), rc, distance, fuel_used, temp, maintenance_flag])

## Create and Inspect DataFrame

Convert the generated list into a pandas DataFrame and inspect the results.

In [18]:
# --- 5. Create DataFrame ---
df = pd.DataFrame(data, columns=[
    "date", "railcar_id", "distance_km", "fuel_used_l",
    "temperature_c", "maintenance_flag"
])

# --- 6. Inspect Dataset ---
print("Preview of Simulated Data:")
display(df.head())

print("\nDataset Summary:")
df.info()

print("\nStatistical Overview:")
display(df.describe())

Preview of Simulated Data:


,date,railcar_id,distance_km,fuel_used_l,temperature_c,maintenance_flag
0,2025-01-01,RC001,194.17,127.73,30.3,0
1,2025-01-01,RC002,62.11,38.11,38.0,0
2,2025-01-01,RC003,184.51,129.84,27.4,0
3,2025-01-01,RC004,139.35,108.61,18.4,0
4,2025-01-01,RC005,135.79,104.51,27.9,0



Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              900 non-null    object 
 1   railcar_id        900 non-null    object 
 2   distance_km       900 non-null    float64
 3   fuel_used_l       900 non-null    float64
 4   temperature_c     900 non-null    float64
 5   maintenance_flag  900 non-null    int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 42.3+ KB

Statistical Overview:


,distance_km,fuel_used_l,temperature_c,maintenance_flag
count,900.000000,900.000000,900.000000,900.000000
mean,122.387533,85.342100,27.433444,0.022222
std,44.261087,31.473331,7.023626,0.147488
min,50.040000,31.060000,15.000000,0.000000
25%,84.212500,59.717500,21.775000,0.000000
50%,120.035000,82.190000,27.300000,0.000000
75%,162.445000,110.852500,33.225000,0.000000
max,199.980000,156.540000,40.000000,1.000000


## Validate Dataset

Check total records, number of unique railcars, and ensure no missing values exist.

In [16]:
# --- 7. Verify Row Count ---
unique_cars = df["railcar_id"].nunique()
rows = len(df)
print(f"\nTotal Records: {rows}  |  Unique Railcars: {unique_cars}")


Total Records: 900  |  Unique Railcars: 15


## Export Dataset to CSV

Finally, we’ll save the dataset into a clean `data/raw/` folder, ready to upload to **Azure Blob Storage** in Stage 2.


In [17]:
# --- 8. Save to CSV ---
os.makedirs("data/raw", exist_ok=True)

csv_path = "data/raw/railcar_data.csv"
df.to_csv(csv_path, index=False)

print(f"\n Dataset saved successfully to: {os.path.abspath(csv_path)}")


 Dataset saved successfully to: C:\Users\j1wad\OneDrive - McMaster University\RailTrack-Cloud\notebooks\data\raw\railcar_data.csv
